In [ ]:
import requests
import pandas as pd
from pyspark.sql import SparkSession as ss
from pyspark.sql.functions import col,year, month, day, dayofweek, regexp_replace,lit,when,udf,current_date,expr,date_sub,to_timestamp,to_date,date_format
from pyspark.sql.types import TimestampType,IntegerType

In [ ]:
spark= ss.builder.appName("linkedin_post_analysis").getOrCreate()
user_name=dbutils.secrets.get(scope="db_scope",key="usesr_name")
password=dbutils.secrets.get(scope="db_scope",key="password")
options={
    "sfURL":"https://cd04219.central-india.azure.snowflakecomputing.com",
    "sfUser":user_name,
    "sfPassword":password,
    "sfDatabase":'JOBPOSTDB',
    "sfWarehouse":'DATAVISTA',
    "sfRole":'SYSADMIN',
    
}

In [ ]:

url = "https://rapid-linkedin-jobs-api.p.rapidapi.com/search-jobs-v2"

querystring = {"keywords":"Software Engineer","locationId":"102713980","datePosted":"pastMonth","sort":"mostRelevant"}

headers = {
	"x-rapidapi-key": "05dd045b2emshb227b9e8c5e9bbcp11f32ejsn1c2407ef9800",
	"x-rapidapi-host": "rapid-linkedin-jobs-api.p.rapidapi.com"
}
try:
	response = requests.get(url, headers=headers, params=querystring)
	json_file=response.json()
except requests.exceptions.RequestExcepion as e:
    print(f"An error occurred during the API request: {e}")
    json_file = None

if json_file:
    try:
        pdf = pd.json_normalize(json_file['data'])
        df=spark.createDataFrame(pdf)
        df=df.filter(col('id').isNotNull())
    except Exception as e:
        print(f"An error occurred during the API request: {e}")
else:
    print("No data available from the API.")
       
    

In [ ]:
df.count()

50

In [ ]:
df_job_postings = df.select(
    col('id'),
    col('referenceId'),
)
df_job_postings=df_job_postings.withColumn('id',col('id').cast('long'))
display(df_job_postings)

id,referenceId
3990999045,+XyhVw1DBgffGIhFglsOAg==
3850652988,+XyhVw1DBgffGIhFglsOAg==
3784329235,+XyhVw1DBgffGIhFglsOAg==
3984088810,+XyhVw1DBgffGIhFglsOAg==
3982726787,+XyhVw1DBgffGIhFglsOAg==
3986539089,+XyhVw1DBgffGIhFglsOAg==
3994591494,+XyhVw1DBgffGIhFglsOAg==
568136544,+XyhVw1DBgffGIhFglsOAg==
3986516681,+XyhVw1DBgffGIhFglsOAg==
3984893318,+XyhVw1DBgffGIhFglsOAg==


In [ ]:
df_company=df.select(
    col('id'),
    col('`company.id`').alias('company_id'),
    col('`company.name`').alias('company_name'),
    col('`company.url`').alias("company_url")
)
df_company=df_company.withColumn('id',col('id').cast('long'))
df_company.show()


+----------+------------+--------------------+--------------------+
|        id|  company_id|        company_name|         company_url|
+----------+------------+--------------------+--------------------+
|3990999045| 1.1149985E7| Formula Recruitment|https://www.linke...|
|3850652988|  1.400332E7|          asambeauty|https://www.linke...|
|3784329235|1.00873698E8|    D5 Software GmbH|https://www.linke...|
|3984088810| 4.7672239E7|    Rakuten Symphony|https://www.linke...|
|3982726787|   2667237.0|            YunoJuno|https://www.linke...|
|3986539089|   9233427.0|              THRYVE|https://www.linke...|
|3994591494| 7.8457641E7|DFSC Projekt- & E...|https://www.linke...|
| 568136544| 1.1499752E7|           WAPP GmbH|https://www.linke...|
|3986516681| 8.6691134E7|   Intega Consulting|https://www.linke...|
|3984893318|      1088.0|  NXP Semiconductors|https://www.linke...|
|3996284212| 9.8400737E7|The Associated En...|https://www.linke...|
|3996698257|        NULL|          MateroSoft|  

In [ ]:
df_job=df.select(
    col('id'),
    col('title'),
    col('url').alias('jpb_url'),
    col('type'),
    col('location').alias('job_location')
)
df_job=df_job.withColumn('id',col('id').cast('long'))
display(df_job)

id,title,jpb_url,type,job_location
3990999045,Software Engineer,https://www.linkedin.com/jobs/view/3990999045,Full-time,"Berlin, Germany (Hybrid)"
3850652988,IT System Engineer / Netzwerkadministration (m/w/d),https://www.linkedin.com/jobs/view/3850652988,Full-time,"Unterföhring, Bavaria, Germany (On-site)"
3784329235,Full Stack Engineer,https://www.linkedin.com/jobs/view/3784329235,Full-time,"Düsseldorf, North Rhine-Westphalia, Germany (Hybrid)"
3984088810,Senior Security Infrastructure Engineer (m/f/d),https://www.linkedin.com/jobs/view/3984088810,Full-time,"Düsseldorf, North Rhine-Westphalia, Germany (On-site)"
3982726787,Fullstack Developer,https://www.linkedin.com/jobs/view/3982726787,Contract,"Berlin, Germany (Hybrid)"
3986539089,Senior Devops Engineer,https://www.linkedin.com/jobs/view/3986539089,Full-time,"Berlin, Germany (On-site)"
3994591494,Softwareingenieur (m/w/d),https://www.linkedin.com/jobs/view/3994591494,Full-time,"Pirmasens, Rhineland-Palatinate, Germany (Hybrid)"
568136544,"Fullstack-Entwickler:in (React, Kotlin/Java)",https://www.linkedin.com/jobs/view/568136544,Full-time,"Bielefeld, North Rhine-Westphalia, Germany (On-site)"
3986516681,DevOps Engineer,https://www.linkedin.com/jobs/view/3986516681,Full-time,"Kreuzwertheim, Bavaria, Germany (Hybrid)"
3984893318,R-10052989 Senior Software Application Engineer – Ethernet Products (f/m/d),https://www.linkedin.com/jobs/view/3984893318,Full-time,"Hamburg, Germany (Hybrid)"


In [ ]:

def time_ago_to_days(time_ago_str):
    if "day" in time_ago_str:
        return int(time_ago_str.split()[0])
    elif "week" in time_ago_str:
        return int(time_ago_str.split()[0]) * 7
    elif "hour" in time_ago_str:
        return 0  # If it's hours ago, it counts as today
    elif "month" in time_ago_str:
        return int(time_ago_str.split()[0]) * 30  # Approximate to 30 days
    elif "year" in time_ago_str:
        return int(time_ago_str.split()[0]) * 365  # Approximate to 365 days
    else:
        return None 


df_date=df.select(col('id'),col('postDate'))
df_date=df_date.withColumn('id',col('id').cast('long'))
                      
time_ago_udf = udf(time_ago_to_days, IntegerType())    
df_date = df_date.withColumn("date", date_sub(current_date(), time_ago_udf(col("postDate"))))

df_date = df_date.withColumn('year', date_format(col("date"), "yyyy")) \
                 .withColumn("month", date_format(col("date"), "MM")) \
                 .withColumn("day", date_format(col("date"), "dd")) \
                 .withColumn("day_of_week", date_format(col("date"), "EEEE"))

df_date=df_date.select('id','date','year','month','day','day_of_week')

df_date = df_date.withColumn("month", 
                             when(col("month") == 1, "Jan")
                             .when(col("month") == 2, "Feb")
                             .when(col("month") == 3, "Mar")
                             .when(col("month") == 4, "Apr")
                             .when(col("month") == 5, "May")
                             .when(col("month") == 6, "Jun")
                             .when(col("month") == 7, "Jul")
                             .when(col("month") == 8, "Aug")
                             .when(col("month") == 9, "Sep")
                             .when(col("month") == 10, "Oct")
                             .when(col("month") == 11, "Nov")
                             .when(col("month") == 12, "Dec")
                             .otherwise("Unknown"))
df_date.show()

+----------+----------+----+-----+---+-----------+
|        id|      date|year|month|day|day_of_week|
+----------+----------+----+-----+---+-----------+
|3990999045|2024-08-02|2024|  Aug| 02|     Friday|
|3850652988|2024-08-02|2024|  Aug| 02|     Friday|
|3784329235|2024-08-09|2024|  Aug| 09|     Friday|
|3984088810|2024-08-02|2024|  Aug| 02|     Friday|
|3982726787|2024-07-26|2024|  Jul| 26|     Friday|
|3986539089|2024-08-02|2024|  Aug| 02|     Friday|
|3994591494|2024-08-09|2024|  Aug| 09|     Friday|
| 568136544|2024-08-13|2024|  Aug| 13|    Tuesday|
|3986516681|2024-08-02|2024|  Aug| 02|     Friday|
|3984893318|2024-08-02|2024|  Aug| 02|     Friday|
|3996284212|2024-08-09|2024|  Aug| 09|     Friday|
|3996698257|2024-08-09|2024|  Aug| 09|     Friday|
|3994966442|2024-08-09|2024|  Aug| 09|     Friday|
|3993289649|2024-08-09|2024|  Aug| 09|     Friday|
|3983522810|2024-07-26|2024|  Jul| 26|     Friday|
|3983965465|2024-07-26|2024|  Jul| 26|     Friday|
|3987757743|2024-08-02|2024|  A

In [ ]:
try :
    df_company.write \
    .format('snowflake') \
    .options(**options) \
    .option('dbtable','linkedin_job_stg_schema.stg_dim_company') \
    .mode('append') \
    .save()
    print("df_company data successfully written to snowflake")
except Exception as e:
    print("there is a error in df_company",e)



try :
    df_job.write \
    .format('snowflake') \
    .options(**options) \
    .option('dbtable','linkedin_job_stg_schema.stg_dim_job') \
    .mode('append') \
    .save()
    print("df_job data successfully written to snowflake")
except Exception as e:
    print("there is a error in df_job",e)



try :
    df_date.write \
    .format('snowflake') \
    .options(**options) \
    .option('dbtable','linkedin_job_stg_schema.stg_dim_date') \
    .mode('append') \
    .save()
    print("df_date data successfully written to snowflake")
except Exception as e:
    print("there is a error in df_date",e)




try :
    df_job_postings.write \
    .format('snowflake') \
    .options(**options) \
    .option('dbtable','linkedin_job_stg_schema.stg_fact_post') \
    .mode('append') \
    .save()
    print("df_job_postings data successfully written to snowflake")
except Exception as e:
    print("there is a error in df_job_postings",e)

df_company data successfully written to snowflake
df_job data successfully written to snowflake
df_date data successfully written to snowflake
df_job_postings data successfully written to snowflake
